# Autoencoder Simplificado para o Dataset Fashion MNIST

Neste notebook, vamos construir um autoencoder simplificado para o dataset Fashion MNIST. O objetivo é aprender a representação compacta dos dados e, em seguida, usar essa representação para reconstruir os dados originais.

O notebook está estruturado da seguinte maneira:

1. Análise Exploratória de Dados (EDA)
2. Construção e Treinamento do Autoencoder
3. Visualização do Resultado do Decoder
4. Construção e Comparação de Classificadores RandomForest e XgBoost
5. Visualização de Dados com TSNE
6. Repetição do Processo com um Autoencoder Convolucional
7. Comparação dos Resultados (loss) do Autoencoder Simplificado e Convolucional
8. Utilização da Última Camada de Max Pooling como Feature para um Classificador
9. Visualização de Dados com TSNE

In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.manifold import TSNE
from xgboost import XGBClassifier

import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

## 1. Análise Exploratória de Dados (EDA)

Vamos começar carregando o dataset Fashion MNIST e realizando uma análise exploratória básica para entender melhor os dados.

In [ ]:
# Carregando o dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalizando os valores dos pixels para o intervalo [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape dos dados para o formato (n_samples, n_features)
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Print das dimensões dos datasets
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

## 2. Construção e Treinamento do Autoencoder

Agora, vamos construir e treinar o autoencoder. Para isso, vamos definir a arquitetura do modelo, compilar o modelo e, finalmente, treinar o modelo.

In [ ]:
# Definindo a dimensão do código codificado
encoding_dim = 32

# Definindo a camada de entrada
input_img = Input(shape=(784,))

# Definindo a camada de codificação
encoded = Dense(encoding_dim, activation='relu')(input_img)

# Definindo a camada de decodificação
decoded = Dense(784, activation='sigmoid')(encoded)

# Definindo o autoencoder
autoencoder = Model(input_img, decoded)

# Definindo o codificador
encoder = Model(input_img, encoded)

# Definindo o decodificador
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

# Compilando o autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Treinando o autoencoder
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

## 3. Visualização do Resultado do Decoder

Agora que o autoencoder foi treinado, vamos visualizar o resultado do decoder. Para isso, vamos codificar e decodificar algumas imagens do conjunto de teste e comparar as imagens originais com as imagens reconstruídas.

In [ ]:
# Codificando e decodificando algumas imagens do conjunto de teste
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# Plotando as imagens originais e as imagens reconstruídas
n = 10  # número de imagens para exibir
plt.figure(figsize=(20, 4))
for i in range(n):
    # imagens originais
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # imagens reconstruídas
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## 4. Construção e Comparação de Classificadores RandomForest e XgBoost

Vamos agora construir dois classificadores, um usando o algoritmo RandomForest e outro usando o algoritmo XgBoost. Depois de treinados, vamos comparar o desempenho de ambos.

In [ ]:
# Construindo e treinando o classificador RandomForest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED)
rf_classifier.fit(x_train, y_train)

# Avaliando o classificador RandomForest
rf_predictions = rf_classifier.predict(x_test)
print('RandomForest Classifier:
', classification_report(y_test, rf_predictions))

# Construindo e treinando o classificador XgBoost
xgb_classifier = XGBClassifier(random_state=RANDOM_SEED)
xgb_classifier.fit(x_train, y_train)

# Avaliando o classificador XgBoost
xgb_predictions = xgb_classifier.predict(x_test)
print('XgBoost Classifier:
', classification_report(y_test, xgb_predictions))

## 5. Visualização de Dados com TSNE

Vamos agora realizar uma visualização de dados utilizando o algoritmo TSNE. Este algoritmo é útil para visualizar dados de alta dimensão em um espaço de duas ou três dimensões.

In [ ]:
# Reduzindo a dimensionalidade dos dados com TSNE
tsne = TSNE(n_components=2, random_state=RANDOM_SEED)
x_test_2D = tsne.fit_transform(x_test)

# Plotando os dados
plt.figure(figsize=(10, 8))
scatter = plt.scatter(x_test_2D[:, 0], x_test_2D[:, 1], c=y_test, cmap='viridis')
plt.colorbar(scatter)
plt.title('Visualização de Dados com TSNE')
plt.show()

## 6. Repetição do Processo com um Autoencoder Convolucional

Vamos agora repetir o processo, mas desta vez utilizando um autoencoder convolucional. Os autoencoders convolucionais são especialmente eficazes quando lidamos com imagens.

In [ ]:
# Carregando o dataset novamente
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalizando os valores dos pixels para o intervalo [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Adicionando uma dimensão extra para os canais
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Print das dimensões dos datasets
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
# Definindo a arquitetura do autoencoder convolucional
input_img = Input(shape=(28, 28, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Treinando o autoencoder convolucional
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

## 7. Comparação dos Resultados (loss) do Autoencoder Simplificado e Convolucional

Agora que treinamos ambos os autoencoders, podemos comparar seus resultados. Para isso, vamos calcular a perda (loss) de ambos os modelos no conjunto de teste.

In [ ]:
# Calculando a perda do autoencoder simplificado
simplified_autoencoder_loss = autoencoder.evaluate(x_test.reshape(-1, 784), x_test.reshape(-1, 784), verbose=0)

# Calculando a perda do autoencoder convolucional
convolutional_autoencoder_loss = autoencoder.evaluate(x_test, x_test, verbose=0)

print(f'Perda do Autoencoder Simplificado: {simplified_autoencoder_loss}')
print(f'Perda do Autoencoder Convolucional: {convolutional_autoencoder_loss}')

## 8. Utilização da Última Camada de Max Pooling como Feature para um Classificador

Vamos agora tentar utilizar a última camada de max pooling do autoencoder convolucional como feature para um classificador. Para isso, vamos extrair as features dessa camada, treinar um classificador com essas features e avaliar o desempenho do classificador.

In [ ]:
# Extraindo as features da última camada de max pooling
feature_extractor = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('max_pooling2d_2').output)
x_train_features = feature_extractor.predict(x_train)
x_test_features = feature_extractor.predict(x_test)

# Redimensionando as features para o formato (n_samples, n_features)
x_train_features = x_train_features.reshape((len(x_train_features), np.prod(x_train_features.shape[1:])))
x_test_features = x_test_features.reshape((len(x_test_features), np.prod(x_test_features.shape[1:])))

# Construindo e treinando o classificador RandomForest com as features extraídas
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED)
rf_classifier.fit(x_train_features, y_train)

# Avaliando o classificador RandomForest
rf_predictions = rf_classifier.predict(x_test_features)
print('RandomForest Classifier:
', classification_report(y_test, rf_predictions))

## 9. Visualização de Dados com TSNE (utilizando as features extraídas)

Por fim, vamos realizar novamente uma visualização de dados utilizando o algoritmo TSNE, mas desta vez utilizando as features extraídas da última camada de max pooling do autoencoder convolucional.

In [ ]:
# Reduzindo a dimensionalidade das features com TSNE
x_test_features_2D = tsne.fit_transform(x_test_features)

# Plotando os dados
plt.figure(figsize=(10, 8))
scatter = plt.scatter(x_test_features_2D[:, 0], x_test_features_2D[:, 1], c=y_test, cmap='viridis')
plt.colorbar(scatter)
plt.title('Visualização de Dados com TSNE (utilizando as features extraídas)')
plt.show()